# A cryptocurrency from scratch

When Bitcoin exploded in december 2017, it blew all expectations of what a cryptocurrency could do. It became a mainstream currency and an investement hype, which led to the creation of thousands of new cryptocurrencies. What I find so fascinating is that it's a purely mathematical system, which relies on the fact that we can't solve these problems.

I will try to build to build a cryptocurrency from scratch. I'll use the [fantastic blog post](http://karpathy.github.io/2021/06/21/blockchain/) of Andrej Karpathy as a guide.

## 1. Generating a crypto identity

We want to be able to generate a crypto identity, so a public and a private key we can use for encryption. In Andrej Karpathy's blog post, he uses elliptic curve cryptography, which are used in bitcoins. I don't know much about the topic, so I decided to use the more standard method of RSA. 

The keys are generated in 5 steps:

### 1. create two large prime number p and q

We will generate two large number randomly, and then test if they are prime with the Miller-Rabin test. We will need the following functions:

In [1]:
def millerRabinTest(n, k):
    def isComposite(a, d, n):
        x = pow(a, d, n)
        
        if x == 1 or x == n-1: 
          return False
        
        for r in range(1, s):
            x = pow(x, 2, n)
            if x == 1: 
              return True
            if x == n - 1: 
              return False
        
        return True
    
    s = 0
    d = n - 1

    while d % 2 == 0:
      s += 1
      d //= 2

    for i in range(k):
        a = random.randint(2, n - 1)
        if isComposite(a, d, n):
          return False
    
    return True

The Miller-Rabin test can't exactly say if a number is prime. It will insted try to find a witness for the composition of the canditate n. If we find one, we can certainly say that the number is not prime, if not it may be prime. The more time we run the algorithm (using the constant k), the higher the probability it's prime.

In [2]:
import random

def generatePrime(keyLength, k):
  while True:
    n = random.randint(2**(keyLength - 1), 2**(keyLength))
    t = millerRabinTest(n, k)
    
    if t:
      return n

In [3]:
print("prime: " + str(generatePrime(25, 20)))

prime: 17241127


### 2. calculate the modulus of the public and private key

The modulus n is the product of the public key p and the private key q, and will be released as part of a public key.

In [4]:
def product(p, q):
  n = p * q
  return n

In [5]:
p = generatePrime(25, 20)
q = generatePrime(25, 20)
n = product(p, q)
print("p = " + str(p))
print("q = " + str(q))
print("n = " + str(n))

p = 18745351
q = 28902611
n = 541789588011461


### 3. calculate the Carmichael function

The Camichael function associates to every positive integer n, a positive integer m, defined as the smallest integer such that 

\begin{equation*} a^m ≡ 1   (mod\ n) \end{equation*}

We can find it by using the least common multiplier, which intern we can find with the greatest common multiplier.

In [6]:
#greatest common multiplier

def gcd(a, b):
  while b != 0:
    a, b = b, a%b
  return a

In [7]:
#least common multiplier

def lcm(a, b):
  return int(abs(a * b) / gcd(a, b))

In [8]:
# carmicheal totient function

def carmichael(p, q):
  l = lcm(p - 1, q - 1)
  return l

In [9]:
l = carmichael(p, q)
print("Carmichael(" + str(p) + ", " + str( q) + ") = " + str(l))

Carmichael(18745351, 28902611) = 54178954036350


### 4. find a number e, coprime to l

We want to find an integer e, such that 1 < e < l and gcd(e, l) = 1.

In [10]:
def coprime(l):
  while (True):
    e = random.randint(2, l)
    if (gcd(e, l) == 1):
       return e

In [11]:
e = coprime(l)
print("e = " + str(e) + " is coprime to l = " + str(l))

e = 50770380771443 is coprime to l = 54178954036350


### 5. calculate d, the modular inverse of e

To find the modular inverse of e, we can use the extended euclidean algorithm. Which find s and t, such that \begin{equation*} a * s + b * t = gcd(a, b) \end{equation*}

In [12]:
#extended euclidian algorithm

def extendedEuclid(a, b):
  (s, ns) = (1, 0)
  (t, nt) = (0, 1)

  while b != 0:
    q = (a // b)
    (a, b) = (b, a - q * b)
    (s, ns) = (ns, s - q * ns)
    (t, nt) = (nt, t - q * nt)
  
  return a, s, t

In [13]:
# the modular inverse

def modularInverse(e, l):
  gcd, s, t = extendedEuclid(e, l)

  return (s % l)

In [14]:
d = modularInverse(e, l)
print("d = " + str(d) + " is the modular inverse of e = " + str(e) + " mod l = " + str(l))
print("because d*e mod l = " + str((d * e) % l))

d = 28851178150007 is the modular inverse of e = 50770380771443 mod l = 54178954036350
because d*e mod l = 1


### generating the key pair

The pairs are composed of (n, e) for the public key and (n, d) for the secret key.

In [15]:
def getKeyPair(keyLength, strength):
  p = generatePrime(keyLength, strength)
  q = generatePrime(keyLength, strength)

  n = product(p, q)
  l = carmichael(p, q)

  e = coprime(l)
  publicKey = (n, e)
  
  d = modularInverse(e, l)
  secretKey = (n, d)

  return publicKey, secretKey

Let's generate a key pair and use it for encryption.

In [40]:
import math

pk, sk = getKeyPair(25, 20)

print("public key", pk)
print("secret key", sk)
print("")

def encrypt(msg, pk):
  (n, e) = pk
  cyf = pow(m, e, n)
  return cyf

def decrypt(cyf, sk):
  (n, d) = sk
  m = pow(cyf, d, n)
  return m

m = 30412312233

c = encrypt(m, pk)
nm = decrypt(c, sk)

print(str(m) + " -> encrypt -> " + str(c) + " -> decrypt -> " + str(nm))

public key (557429653189553, 285536948875)
secret key (557429653189553, 78387856531)

30412312233 -> encrypt -> 101551008830620 -> decrypt -> 30412312233


We have some constraint in this system. RSA can't encrypt messages which are bigger than the modulus n. Python seems also to have problem with larger expontentials, so we are limited to size 25... A better way would be to use crypto libraries, but the target is not create a real cryptocurrency, but look at the math behind.

## 2. create the hashing methods

Central to cryptocurrencies are cryptographic hash function, they are one way functions who map an input to a fixed size output. Bitcoin uses SHA256 and Andrej Karpathy goes on to implemented it from scratch. It's basically just a bunch of binary operation. 

We will use djb2, a simpler hash algorithm, which still works very good.

In [80]:
def string2bits(s):
    return "".join([bin(ord(x))[2:].zfill(8) for x in s])

def bits2string(s):
    return ''.join([chr(int(x, 2)) for x in s.split(4)])

def hash(st):
  print(type(st))
  st = str.encode(st)
  for s in st:
    s = (s << 5)
  return st

hash("hello world")

<class 'str'>


b'hello world'

<class 'str'>


b'hello world'